In [1]:
import os

os.chdir("/home/tim/Development/OCPPM/")
import pandas as pd
import pm4py
import pm4py.read
import category_encoders as ce

In [2]:
ocel_in_file = "data/CS/source/cs_january_ots[krs_krv_cv]_eas[ea2_ea4_resource_ce_resource_multi].jsonocel"
objects_table_in_file = "data/CS/source/objects.csv"

oft_out_file = "data/CS/feature_encodings/baselines/OFT/objects.csv"

### TODO: use same object attributes as in actual OCEL (to be able to compare against EFG)

In [3]:
# load OCEL
ocel = pm4py.read.read_ocel(ocel_in_file)
objects_df = pd.read_csv(objects_table_in_file, sep=";")

In [4]:
# select events that contain the target variable 'ea4'
events_to_take_target_from = (
    ocel.relations.sort_values("ocel:timestamp", ascending=False)
    .groupby("ocel:oid")
    .tail(1)[["ocel:eid", "ocel:oid"]]
)
# select the target variables for those events
events_with_target = ocel.events.loc[
    ocel.events["ocel:eid"].isin(events_to_take_target_from["ocel:eid"]),
    ["ocel:eid", "event_ea4"],
]
# select the objects relating to those events and their target variables
objects_with_target = events_to_take_target_from.join(
    events_with_target.set_index("ocel:eid"), on="ocel:eid"
).drop(columns=["ocel:eid"])
# join full objects table with these objects of which we now have target variable 'ea4'
objects_df_with_target = objects_with_target.join(
    objects_df.set_index("oid"), on="ocel:oid"
)

In [6]:
cat_cols = [
    k
    for (k, v) in objects_df_with_target.dtypes.to_dict().items()
    if (k != "ocel:oid") and (v == object)
]

In [7]:
processed_objects_df = objects_df_with_target.drop(columns=["event_ea4", "ocel:oid"])

encoder = ce.CountEncoder(cols=cat_cols)
processed_objects_df = encoder.fit_transform(processed_objects_df)
processed_objects_df["ea4"] = objects_df_with_target["event_ea4"]
processed_objects_df["oa14"] = processed_objects_df["oa14"].fillna(0)

In [8]:
processed_objects_df.to_csv(oft_out_file, index=False, sep=";")